In [1]:
!pip install mne --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 81.0 MB/s eta 0:00:00


In [2]:
# Uninstall existing PyTorch
!pip uninstall torch torchvision torchaudio -y

# Install the stable version for CUDA 11.8 (Adjust if using a different CUDA version)
!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.6/811.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 126.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/7

In [3]:
!git clone https://github.com/staraink/MIRepNet.git


Cloning into 'MIRepNet'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 331 (delta 36), reused 15 (delta 15), pack-reused 279 (from 2)
Receiving objects: 100% (331/331), 25.03 MiB | 22.07 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [ ]:
!pip install numpy==1.26.4

In [1]:
import numpy as np
import mne
import os
from pathlib import Path
from glob import glob
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
import gc
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# =========================================================================
# I. CONFIGURATION
# =========================================================================

TOTAL_SUBJECTS = 109
TARGET_SFREQ = 128.0
base_data_path = Path('/content/drive/MyDrive/EEG_Processed_Data')

# Runs Configuration
runs_in_order = [7, 8, 9, 10, 13, 14]
hand_runs = [7, 8, 13, 14]
feet_runs = [9, 10]
expected_num_runs = len(runs_in_order)
event_id_4class = { 'Rest':0, 'Left':1, 'Right':2, 'Forward':3 }
min_segment_len = 5

# =========================================================================
# II. MAIN PROCESSING LOOP: Load, Epoch, and Collect NumPy Arrays (Memory Optimized)
# =========================================================================

all_X = []
all_y = []
total_epochs_count = 0

print(f"Starting memory-optimized loading and epoching, TARGET SFREQ: {TARGET_SFREQ}Hz...")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for i in range(1, TOTAL_SUBJECTS + 1):
        subject_name = f'S{i:03d}'
        subject_folder = base_data_path / subject_name
        cleaned_file = subject_folder / f"{subject_name}_cleaned_raw.fif"

        if not cleaned_file.exists():
            if i % 10 == 1:
                print(f"[SKIP] {subject_name}: cleaned file not found at {cleaned_file}")
            continue

        try:
            combined_raw = mne.io.read_raw_fif(cleaned_file, preload=True, verbose=False)
        except Exception as e:
            print(f"[ERROR] {subject_name}: failed to read {cleaned_file}: {e}")
            continue

        # --- Resampling ---
        if combined_raw.info['sfreq'] != TARGET_SFREQ:
            combined_raw.resample(TARGET_SFREQ, npad='auto', verbose=False)
        sfreq = combined_raw.info['sfreq']

        # --- Epoching Logic (omitted for brevity, remains the same) ---
        annotations = combined_raw.annotations
        if annotations is None or len(annotations) == 0:
            del combined_raw; gc.collect(); continue
        try:
            events, event_dict = mne.events_from_annotations(combined_raw)
        except Exception as e:
            del combined_raw; gc.collect(); continue

        # Run Segmentation (merge boundary sections)
        boundary_labels = {'BAD boundary', 'EDGE boundary'}
        desc_list = [str(a['description']) for a in annotations]
        boundary_idx = [idx for idx, d in enumerate(desc_list) if d in boundary_labels]

        segment_edges = [-1] + boundary_idx + [len(desc_list) - 1]
        segments = []
        for s in range(len(segment_edges) - 1):
            start = segment_edges[s] + 1
            end = segment_edges[s + 1]
            if start <= end:
                segments.append((start, end))

        merged = []
        for s, e in segments:
            length = e - s + 1
            if len(merged) > 0 and length < min_segment_len:
                last_s, last_e = merged.pop()
                merged.append((last_s, e))
            else:
                merged.append((s, e))
        segments = merged

        if len(segments) != expected_num_runs:
            if i % 10 == 0:
                print(f"[SEGMENT FAIL] {subject_name}: found {len(segments)} segments (expected {expected_num_runs}) - skipping")
            del combined_raw; gc.collect(); continue

        full_run_list = np.empty(len(desc_list), dtype=int)
        for seg_idx, (s, e) in enumerate(segments):
            runnum = runs_in_order[seg_idx]
            full_run_list[s:e+1] = runnum

        # Context-Dependent Event Mapping (T0/T1/T2 -> 0/1/2/3)
        mapped_events = events.copy()

        for ann_idx, ann in enumerate(annotations):
            desc = str(ann['description'])
            try:
                run = int(full_run_list[ann_idx])
            except Exception:
                continue

            if desc not in ('T0', 'T1', 'T2'):
                continue

            onset_sec = ann['onset']
            sample = int(round(onset_sec * sfreq))

            if desc == 'T0':
                new_val = 0
            elif desc == 'T1':
                new_val = 1
            elif desc == 'T2':
                new_val = 2 if run in hand_runs else 3
            else:
                continue

            close_idx = np.where(np.abs(events[:, 0] - sample) <= 1)[0]
            if close_idx.size > 0:
                mapped_events[close_idx, 2] = new_val
            else:
                nearest_idx = np.argmin(np.abs(events[:, 0] - sample))
                mapped_events[nearest_idx, 2] = new_val

        # 4. Create Epochs, Drop Bad, and COLLECT NUMPY ARRAYS
        try:
            epochs_subject = mne.Epochs(
                combined_raw, mapped_events, event_id=event_id_4class,
                tmin=-0.2, tmax=4.0, baseline=(-0.2, 0), preload=True, verbose=False
            )
        except Exception as e:
            print(f"[ERROR] {subject_name}: creating Epochs failed: {e}")
            del combined_raw; gc.collect(); continue

        try:
            epochs_subject.drop_bad()
        except Exception:
            pass

        num_epochs = len(epochs_subject)
        if num_epochs > 0:
            # 🚨 CRITICAL OPTIMIZATION: Convert to float32 here to halve memory!
            X_data = epochs_subject.get_data().astype(np.float32)
            all_X.append(X_data)
            all_y.append(epochs_subject.events[:, 2])
            total_epochs_count += num_epochs

        # clean up large objects & force GC
        del combined_raw
        del epochs_subject
        gc.collect()

        if i % 10 == 0:
            print(f"✅ {subject_name} processed. Total epochs collected so far: {total_epochs_count}")

# =========================================================================
# III. FINAL CONCATENATION AND VERIFICATION (Single Stack)
# =========================================================================

print("\n--- FINAL STEP: Stacking all subjects' data ---")

if total_epochs_count == 0:
    raise RuntimeError("No epochs were collected. Check file paths and loading.")

X_final = np.concatenate(all_X, axis=0)
y_final = np.concatenate(all_y, axis=0)

# Free up lists
del all_X, all_y
gc.collect()

# Verification prints (unchanged)
expected_times = int(np.round((4.0 - (-0.2)) * TARGET_SFREQ)) + 1
print(f"\n🎉 Total Final Epochs Collected: **{X_final.shape[0]}**")
print(f"\n🎉 Total Final Epochs Collected: **{X_final.shape[0]}**")

class_counts = Counter(y_final)
print("Epochs per class (Final Dataset):", class_counts)

print("\n| Class ID | Class Name | Epoch Count |")
print("| :--- | :--- | :--- |")
print(f"| 0 | Rest | {class_counts.get(0, 0)} |")
print(f"| 1 | Left | {class_counts.get(1, 0)} |")
print(f"| 2 | Right | {class_counts.get(2, 0)} |")
print(f"| 3 | Forward | {class_counts.get(3, 0)} |")
print(f"\nFinal data shape: **{X_final.shape}** (Epochs, Channels, Times)")
print(f"Expected number of time points at 128Hz: {expected_times}")

# -------------------------------------------------------------------------
# IV. SPLITTING AND STANDARDIZATION (MAX MEMORY OPTIMIZED)
# -------------------------------------------------------------------------

## 🧠 Splitting and Standardization (MAX Memory Optimized)

# --- 1. Data Splitting (Train/Validation/Test) ---
# Splitting inherently creates three copies, which is unavoidable.
X_train_temp, X_test, y_train_temp, y_test = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42, stratify=y_final
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_temp, y_train_temp, test_size=0.25, random_state=42, stratify=y_train_temp
)

# Delete large intermediate copies immediately
del X_final, y_final, X_train_temp, y_train_temp
gc.collect()

print(f"\n--- Data Split Shapes ---")
print(f"Train shapes: X={X_train.shape}, y={y_train.shape}")
print(f"Validation shapes: X={X_val.shape}, y={y_val.shape}")
print(f"Test shapes: X={X_test.shape}, y={y_test.shape}")


# --- 2. Scaling (Standardization per Channel-Time Point) ---
scaler = StandardScaler()

def scale_and_cleanup(X_data, scaler, fit=False):
    """
    Applies scaling and converts the array to a PyTorch tensor.
    Returns a tensor shaped (N, T, C) where:
      N = epochs, T = timesteps, C = channels/features.
    """
    # X_data: (N, C, T) float32
    N, C, T = X_data.shape

    # 1. Reshape for scaling: (N, C, T) -> transpose to (N, T, C) -> reshape to (N*T, C)
    X_for_scaler = np.ascontiguousarray(X_data).transpose(0, 2, 1).reshape(-1, C)  # shape (N*T, C)

    # 2. Fit/Transform
    if fit:
        X_scaled_flat = scaler.fit_transform(X_for_scaler)
    else:
        X_scaled_flat = scaler.transform(X_for_scaler)

    # 3. Reshape back to (N, T, C)
    X_scaled = X_scaled_flat.reshape(N, T, C)

    # 4. Convert to PyTorch Tensor with dtype float32
    X_tensor = torch.as_tensor(X_scaled, dtype=torch.float32)

    # 5. Cleanup
    del X_for_scaler, X_scaled_flat, X_scaled
    gc.collect()

    return X_tensor  # shape (N, T, C)


# Process Training Data
X_train_tensor = scale_and_cleanup(X_train, scaler, fit=True)
del X_train
gc.collect()

# Process Validation Data
X_val_tensor = scale_and_cleanup(X_val, scaler, fit=False)
del X_val
gc.collect()

# Process Test Data
X_test_tensor = scale_and_cleanup(X_test, scaler, fit=False)
del X_test
gc.collect()

# Convert y arrays to tensors after X data is processed
y_train_tensor = torch.as_tensor(y_train, dtype=torch.long)
y_val_tensor = torch.as_tensor(y_val, dtype=torch.long)
y_test_tensor = torch.as_tensor(y_test, dtype=torch.long)

del y_train, y_val, y_test
gc.collect()

# =========================================================================
# V. DATA BALANCING (Weighted Random Sampler)
# =========================================================================

# The counts are taken directly from your output:
class_counts = Counter({0: 9184, 1: 4896, 2: 3254, 3: 1634})
total_epochs = sum(class_counts.values()) # 18968

# --- 1. Calculate Inverse Frequency Weights ---
# The weight for a class is inversely proportional to its frequency.
# weight_i = 1.0 / count_i
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}

# --- 2. Map Weights to Every Sample in the Training Set ---
# Create a list of weights corresponding to the label of every epoch in y_train_tensor
sample_weights = y_train_tensor.clone().detach().cpu().numpy() # Convert labels to numpy array

# Replace the label index (0, 1, 2, 3) with its corresponding calculated weight
for cls, weight in class_weights.items():
    sample_weights[sample_weights == cls] = weight

# Convert weights back to a PyTorch tensor
sample_weights = torch.from_numpy(sample_weights).double()

# --- 3. Create Weighted Random Sampler ---
# The number of samples to draw in one pass (num_samples) is set to the size
# of the training dataset to ensure a full epoch run.
sampler = torch.utils.data.WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights), # The number of elements to draw (equals dataset size)
    replacement=True # Must be True for oversampling the minority classes
)

print("\n✅ Weighted Random Sampler created for balanced training.")


# --- 3. Convert to PyTorch Tensors and DataLoaders ---
BATCH_SIZE = 64

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

NUM_WORKERS = 4  # adjust to your environment; 0 on Windows without fork

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler,       # use sampler for balanced sampling
    shuffle=False,         # must be False when sampler is provided
    drop_last=True,
    pin_memory=True,
    num_workers=NUM_WORKERS
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
    num_workers=NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
    num_workers=NUM_WORKERS
)

print("\nPyTorch DataLoaders created.")
print("The script is maximally memory efficient for an in-memory solution. Ready for model training.")


Starting memory-optimized loading and epoching, TARGET SFREQ: 128.0Hz...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
✅ S010 processed. Total epochs collected so far: 1740
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descri

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import wandb
import math

class PatchEmbedding(nn.Module):
    def __init__(self, embed_dim=128, num_channels=64, input_timesteps=None):
        super().__init__()

        self.num_channels = num_channels
        self.embed_dim = embed_dim
        self.input_timesteps = input_timesteps

        # Adaptive convolution parameters based on input size
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(1, 25), stride=(1, 1), padding=(0, 12))
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(self.num_channels, 1), stride=(1, 1))
        self.bn = nn.BatchNorm2d(128)
        self.elu = nn.ELU()

        # Adaptive pooling - we'll calculate the kernel size dynamically
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, None))  # Keep temporal dimension flexible
        self.dropout = nn.Dropout(0.5)

        self.projection = nn.Sequential(
            nn.Conv2d(128, embed_dim, (1, 1), stride=(1, 1)),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: (B, T, C) -> we need (B, 1, C, T)
        x = x.unsqueeze(1)  # (B, 1, C, T)
        B, _, C, T = x.size()

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.elu(x)

        # Use adaptive pooling instead of fixed pooling
        x = self.adaptive_pool(x)
        x = self.dropout(x)

        x = self.projection(x)
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size, num_heads, dropout):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.keys = nn.Linear(emb_size, emb_size)
        self.queries = nn.Linear(emb_size, emb_size)
        self.values = nn.Linear(emb_size, emb_size)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        queries = rearrange(self.queries(x), "b n (h d) -> b h n d", h=self.num_heads)
        keys = rearrange(self.keys(x), "b n (h d) -> b h n d", h=self.num_heads)
        values = rearrange(self.values(x), "b n (h d) -> b h n d", h=self.num_heads)
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)
        scaling = self.emb_size ** (1 / 2)
        att = F.softmax(energy / scaling, dim=-1)
        att = self.att_drop(att)
        out = torch.einsum('bhal, bhlv -> bhav', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size, expansion, drop_p):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )

class TransformerEncoderBlock(nn.Sequential):
    def __init__(self, emb_size, num_heads=8, drop_p=0.5, forward_expansion=4, forward_drop_p=0.5):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, num_heads, drop_p),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            ))
        )

class TransformerEncoder(nn.Sequential):
    def __init__(self, depth, emb_size,dropout=0.5):
        super().__init__(*[TransformerEncoderBlock(emb_size,drop_p=dropout) for _ in range(depth)])


class decoder(nn.Module):
    def __init__(self, emb_size=64, depth=2, pretrain=None,**kwargs):
        super().__init__()
        self.transformer = TransformerEncoder(depth, emb_size)
    def forward(self, x):
        x = self.transformer(x)      # [batch_size, seq_length, emb_size]
        return x

class decoder_fft(nn.Module):
    def __init__(self, emb_size=64, depth=2, pretrain=None,**kwargs):
        super().__init__()
        self.transformer = TransformerEncoder(depth, emb_size)
        self.pro= nn.Linear(emb_size, 3*2)
    def forward(self, x):
        out = self.transformer(x)      # [batch_size, seq_length, emb_size]
        out = self.pro(torch.mean(out, dim=1))  # [batch_size, seq_length, 3*2]
        return out

class mlm_mask(nn.Module):
    def __init__(self, emb_size=128, depth=6, n_classes=2,mask_ratio=0.5, pretrain=None,pretrainmode=False):
        super().__init__()
        self.pretrainmode = pretrainmode
        self.embedding = PatchEmbedding(embed_dim=emb_size)
        self.transformer = TransformerEncoder(depth, emb_size,dropout=0.5)
        self.clshead = nn.Linear(emb_size,n_classes)
        self.mask_ratio = mask_ratio
        if pretrain is not None:
            self.init_from_pretrained(pretrain)

        if self.pretrainmode:
            self.mask_token = nn.Parameter(torch.randn(1, 1, emb_size))
            self.decoder = decoder(emb_size=emb_size, depth=2)

    def random_masking(self, x, mask_ratio=0.5):
        B, L, D = x.shape
        len_keep = int(L * (1 - mask_ratio))

        noise = torch.rand(B, L, device=x.device)
        ids_shuffle = torch.argsort(noise, dim=1)
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).expand(-1, -1, D))

        mask_tokens = self.mask_token.repeat(B, L - len_keep, 1)
        x_masked = torch.cat([x_masked, mask_tokens], dim=1)

        x_masked = torch.gather(x_masked, dim=1, index=ids_restore.unsqueeze(-1).expand(-1, -1, D))

        mask = torch.zeros([B, L], device=x.device)
        mask[:, :len_keep] = 1
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def forward(self, x):
        original_x = self.embedding(x)

        if self.pretrainmode:

            x_masked, mask, ids_restore = self.random_masking(original_x,mask_ratio=self.mask_ratio)

            encoded = self.transformer(x_masked)

            reconstructed = self.decoder(encoded)

            cls_output = self.clshead(torch.mean(encoded, dim=1))

            return cls_output, original_x, reconstructed, None
        else:
            transformed = self.transformer(original_x)
            pooled = torch.mean(transformed, dim=1)
            cls_output = self.clshead(pooled)
            return pooled, cls_output
    def init_from_pretrained(self, pretrained_path, freeze_encoder=False, strict=True):
        pretrained_dict = torch.load(pretrained_path)

        model_dict = self.state_dict()

        pretrained_dict = {k: v for k, v in pretrained_dict.items()
                          if k in model_dict and v.shape == model_dict[k].shape}

        model_dict.update(pretrained_dict)

        self.load_state_dict(model_dict, strict=strict)

        if freeze_encoder:
            for name, param in self.named_parameters():
                if 'embedding' in name or 'transformer' in name:
                    param.requires_grad = False

        print(f"Loaded {len(pretrained_dict)}/{len(model_dict)} parameters from pretrained model")

In [3]:
# --- Essential Imports ---
import torch
import torch.nn as nn
import torch.optim as optim
from huggingface_hub import hf_hub_download
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

# --- Define Constants ---
N_CHANNELS = 64
N_CLASSES = 4
EMB_SIZE = 128
DEPTH = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()


def train(model, train_loader, criterion, optimizer, device, scheduler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)

        _, outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    scheduler.step()

    current_lr = optimizer.param_groups[0]['lr']

    epoch_loss = running_loss / len(train_loader)
    accuracy = correct / total * 100

    return epoch_loss, accuracy, current_lr

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            _, outputs = model(data)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(val_loader)
    accuracy = correct / total * 100
    return epoch_loss, accuracy

In [4]:
# 3.1 Download Pre-trained Weights
weight_path = hf_hub_download(repo_id="starself/MIRepNet", filename="MIRepNet.pth")
print(f"Pre-trained weights downloaded to: {weight_path}")

# 3.2 Instantiate the MIRepNet (mlm_mask)
# Ensure N_CLASSES=4 is passed for your task!
model = mlm_mask(
    emb_size=EMB_SIZE,
    depth=DEPTH,
    n_classes=N_CLASSES,
    pretrainmode=False,
    pretrain=weight_path
).to(device)

print("Model initialized and pre-trained backbone weights loaded.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Pre-trained weights downloaded to: /root/.cache/huggingface/hub/models--starself--MIRepNet/snapshots/9bac0439c0d3e9ffdb40ca675d61a51b439a446e/MIRepNet.pth
Loaded 8/110 parameters from pretrained model
Model initialized and pre-trained backbone weights loaded.


In [6]:
# --- 1. FREEZE THE ENCODER (BACKBONE) ---
for param in model.parameters():
    param.requires_grad = False

# --- UNFREEZE ONLY THE CLASSIFICATION HEAD ---
# The classification head is named 'clshead' in your mlm_mask class definition
for name, param in model.named_parameters():
    if 'clshead' in name:
        param.requires_grad = True

print("✅ Backbone (Embedding & Transformer) frozen.")
print("✅ Classification Head (clshead) is unfrozen and ready for training.")

✅ Backbone (Embedding & Transformer) frozen.
✅ Classification Head (clshead) is unfrozen and ready for training.


In [7]:
# Training hyperparameters
NUM_EPOCHS = 50
LEARNING_RATE = 1e-3  # Smaller LR for fine-tuning
WEIGHT_DECAY = 1e-4

# Since you're only training the classification head, use a slightly higher LR
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

In [9]:
# =========================================================================
# V. DATA BALANCING (Weighted Random Sampler - FIXED)
# =========================================================================

# --- 1. Calculate Class Weights Correctly ---
# Get class counts from the training set only
class_counts = Counter(y_train_tensor.numpy())
print("Training set class distribution:", class_counts)

total_samples = len(y_train_tensor)
num_classes = len(class_counts)

# Calculate weights using inverse frequency (more robust method)
class_weights = {}
for cls in range(num_classes):
    count = class_counts.get(cls, 0)
    if count > 0:
        class_weights[cls] = total_samples / (num_classes * count)
    else:
        class_weights[cls] = 1.0  # fallback for missing classes

print("Class weights:", class_weights)

# --- 2. Create Sample Weights Correctly ---
# Create a weight for each sample in the training set
sample_weights = torch.zeros(len(y_train_tensor), dtype=torch.float32)

for idx, label in enumerate(y_train_tensor):
    sample_weights[idx] = class_weights[label.item()]

# Verify weights are valid
print(f"Sample weights - min: {sample_weights.min():.4f}, max: {sample_weights.max():.4f}, sum: {sample_weights.sum():.4f}")
print(f"Any negative weights: {torch.any(sample_weights < 0)}")
print(f"Any zero weights: {torch.any(sample_weights == 0)}")

# Normalize weights to prevent numerical issues
if sample_weights.sum() > 0:
    sample_weights = sample_weights / sample_weights.sum()
else:
    # Fallback: use uniform weights
    sample_weights = torch.ones(len(y_train_tensor), dtype=torch.float32) / len(y_train_tensor)
    print("⚠️  Using uniform weights as fallback")

print(f"After normalization - min: {sample_weights.min():.4f}, max: {sample_weights.max():.4f}, sum: {sample_weights.sum():.4f}")

# --- 3. Create Weighted Random Sampler ---
sampler = torch.utils.data.WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

print("✅ Weighted Random Sampler created successfully.")

# --- 4. Create DataLoaders with Fixed Sampler ---
BATCH_SIZE = 64

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

NUM_WORKERS = 2  # Reduced for stability

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler,
    shuffle=False,  # Important: shuffle must be False when using sampler
    drop_last=True,
    pin_memory=True,
    num_workers=NUM_WORKERS
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
    num_workers=NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
    num_workers=NUM_WORKERS
)

print("✅ DataLoaders created successfully.")
print(f"Train loader: {len(train_loader)} batches")
print(f"Val loader: {len(val_loader)} batches")
print(f"Test loader: {len(test_loader)} batches")

Training set class distribution: Counter({0: 5510, 1: 2938, 2: 1952, 3: 980})
Class weights: {0: 0.5163339382940109, 1: 0.9683458134785569, 2: 1.4574795081967213, 3: 2.9030612244897958}
Sample weights - min: 0.5163, max: 2.9031, sum: 11380.0000
Any negative weights: False
Any zero weights: False
After normalization - min: 0.0000, max: 0.0003, sum: 1.0000
✅ Weighted Random Sampler created successfully.
✅ DataLoaders created successfully.
Train loader: 177 batches
Val loader: 60 batches
Test loader: 60 batches


In [ ]:
# Ultra-simple training loop
def simple_train():
    model.train()
    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        correct = 0
        total = 0

        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)

            # Forward pass
            _, outputs = model(data)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Stats
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data, labels in val_loader:
                data, labels = data.to(device), labels.to(device)
                _, outputs = model(data)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total
        val_acc = 100 * val_correct / val_total

        print(f'Epoch {epoch+1}: Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

        scheduler.step()

    return model

# Run simple training
print("Starting simplified training...")
trained_model = simple_train()

In [6]:
# --- 2. HEAD TRAINING HYPERPARAMETERS ---
HEAD_EPOCHS = 10    # Short training phase
HEAD_LR = 1e-3      # Higher Learning Rate
STEP_SIZE_HEAD = 5
GAMMA_HEAD = 0.5

optimizer_head = optim.AdamW(model.parameters(), lr=HEAD_LR)
criterion = nn.CrossEntropyLoss()
scheduler_head = optim.lr_scheduler.StepLR(optimizer_head, step_size=STEP_SIZE_HEAD, gamma=GAMMA_HEAD)

print(f"\n--- Starting Head Training ({HEAD_EPOCHS} epochs) ---")

best_val_accuracy_head = 0.0

for epoch in range(1, HEAD_EPOCHS + 1):
    # Training step (using the same 'train' function defined earlier)
    train_loss, train_acc, current_lr = train(
        model, train_loader, criterion, optimizer_head, device, scheduler_head
    )

    # Validation step (using the same 'validate' function defined earlier)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(
        f"HEAD EPOCH {epoch}/{HEAD_EPOCHS} | LR: {current_lr:.6f} "
        f"| Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%"
    )

    if val_acc > best_val_accuracy_head:
        best_val_accuracy_head = val_acc
        torch.save(model.state_dict(), 'best_head_trained_model.pth')

print(f"**Head Training Finished. Best Validation Accuracy: {best_val_accuracy_head:.2f}%**")


--- Starting Head Training (10 epochs) ---
HEAD EPOCH 1/10 | LR: 0.001000 | Train Acc: 46.87% | Val Acc: 48.02%
HEAD EPOCH 2/10 | LR: 0.001000 | Train Acc: 48.47% | Val Acc: 47.92%
HEAD EPOCH 3/10 | LR: 0.001000 | Train Acc: 48.36% | Val Acc: 48.23%
HEAD EPOCH 4/10 | LR: 0.001000 | Train Acc: 48.27% | Val Acc: 48.34%
HEAD EPOCH 5/10 | LR: 0.000500 | Train Acc: 48.31% | Val Acc: 48.66%
HEAD EPOCH 6/10 | LR: 0.000500 | Train Acc: 48.42% | Val Acc: 48.37%
HEAD EPOCH 7/10 | LR: 0.000500 | Train Acc: 48.27% | Val Acc: 48.37%
HEAD EPOCH 8/10 | LR: 0.000500 | Train Acc: 48.51% | Val Acc: 48.50%
HEAD EPOCH 9/10 | LR: 0.000500 | Train Acc: 48.36% | Val Acc: 48.05%
HEAD EPOCH 10/10 | LR: 0.000250 | Train Acc: 48.44% | Val Acc: 48.60%
**Head Training Finished. Best Validation Accuracy: 48.66%**


In [18]:
import torch.optim as optim
import torch.nn as nn
import torch

# --- 1. Unfreeze All Layers ---
# Ensure all parts of the model (Backbone and Head) are trainable
for param in model.parameters():
    param.requires_grad = True

# --- 2. Define Layer Groups and Differential LRs ---

# Hyperparameters for Differential Fine-Tuning
BACKBONE_LR = 1e-5    # Very Low LR for the pre-trained layers (e.g., 0.00001)
HEAD_LR = 5e-5        # Higher LR for the new classification head (e.g., 0.00005)
FULL_EPOCHS = 50      # Run for a sufficient number of epochs
STEP_SIZE_FULL = 10
GAMMA_FULL = 0.5

# Collect parameters for the head and backbone separately
params_to_update = [
    # Group 1: The Classification Head (needs faster learning)
    {'params': model.clshead.parameters(), 'lr': HEAD_LR},

    # Group 2: The rest of the Model (Embedding, Transformer Encoder)
    # Use the lower LR for fine-tuning the pre-trained weights
    {'params': (p for name, p in model.named_parameters() if 'clshead' not in name), 'lr': BACKBONE_LR},
]

# --- 3. Setup Optimizer and Scheduler ---
optimizer_full = optim.AdamW(params_to_update)
criterion = nn.CrossEntropyLoss() # Assuming this was already defined
scheduler_full = optim.lr_scheduler.StepLR(optimizer_full, step_size=STEP_SIZE_FULL, gamma=GAMMA_FULL)

print(f"\n--- Starting Differential Full Fine-Tuning ({FULL_EPOCHS} epochs) ---")
print(f"Backbone LR: {BACKBONE_LR:.1e}, Head LR: {HEAD_LR:.1e}")


--- Starting Differential Full Fine-Tuning (50 epochs) ---
Backbone LR: 1.0e-05, Head LR: 5.0e-05


In [ ]:
# --- 1. Define Optimizer Groups ---
BACKBONE_LR_NEW = 5e-5    # Aggressive LR for pre-trained layers
HEAD_LR_NEW = 1e-4        # Higher LR for the new classification head
FULL_EPOCHS = 70          # Extended training time
BEST_MODEL_PATH = 'best_aggressive_ft_model.pth'

# Ensure the model is moved to device (assumes 'model' and 'device' are defined)
model.to(device)

# Ensure all layers are unfreezed
for param in model.parameters():
    param.requires_grad = True

params_to_update_aggressive = [
    # Group 1: Classification Head (Highest LR)
    {'params': model.clshead.parameters(), 'lr': HEAD_LR_NEW},

    # Group 2: Backbone (Higher Adaptation LR)
    {'params': (p for name, p in model.named_parameters() if 'clshead' not in name), 'lr': BACKBONE_LR_NEW},
]

# --- 2. Setup Optimizer and Scheduler ---
optimizer_aggressive = optim.AdamW(params_to_update_aggressive)
criterion = nn.CrossEntropyLoss()
scheduler_aggressive = optim.lr_scheduler.StepLR(optimizer_aggressive, step_size=10, gamma=0.5)

print(f"\n--- Starting AGGRESSIVE Differential Fine-Tuning ({FULL_EPOCHS} epochs) ---")
print(f"NEW Backbone LR: {BACKBONE_LR_NEW:.1e}, NEW Head LR: {HEAD_LR_NEW:.1e}")


# --- 3. Training Loop ---
best_val_accuracy_full = 0.0

for epoch in range(1, FULL_EPOCHS + 1):
    # Call the user-defined train function
    train_loss, train_acc, current_head_lr = train(
        model, train_loader, criterion, optimizer_aggressive, device, scheduler_aggressive
    )

    # Call the user-defined validate function
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(
        f"FULL EPOCH {epoch}/{FULL_EPOCHS} | Head LR: {current_head_lr:.7f} "
        f"| Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%"
        f"| Train loss: {train_loss:.4f}% | Val loss: {val_loss:.4f}%"
    )

    if val_acc > best_val_accuracy_full:
        best_val_accuracy_full = val_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"**Saved new best model with Val Acc: {best_val_accuracy_full:.2f}%**")

print(f"\nFull Fine-Tuning finished! Final Best Validation Accuracy: {best_val_accuracy_full:.2f}%")

In [ ]:
from sklearn.metrics import confusion_matrix

# --- 5. Final Evaluation on Test Set ---

# Load the best fine-tuned model weights
try:
    model.load_state_dict(torch.load(BEST_MODEL_PATH))
    print("\n✅ Loaded best fine-tuned model for final evaluation.")
except FileNotFoundError:
    print("\n⚠️ Warning: Best fine-tuned model file not found. Evaluating with current model state.")

model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
all_predictions = []
all_labels = []

# Evaluation loop
with torch.no_grad():
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        _, outputs = model(data)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

final_test_loss = test_loss / len(test_loader)
final_test_accuracy = test_correct / test_total * 100
conf_matrix = confusion_matrix(all_labels, all_predictions)

print("\n--- Final Test Results ---")
print(f"Test Loss: {final_test_loss:.4f}")
print(f"Test Accuracy: **{final_test_accuracy:.2f}%**")
print("\nConfusion Matrix:")
print(conf_matrix)